# Camera Acquisition App Description

This Python application provides a GUI-based camera acquisition system using PySpin, OpenCV, and Tkinter. It allows the user to control camera settings such as frame rate (FPS), exposure time, and output file name. The application supports video recording and displays the camera feed with ArUco marker detection.

## Features:
- **Camera Settings**:
  - FPS (frames per second)
  - Exposure time (microseconds)
  - Output file name for saving video
- **ArUco Marker Detection**:
  - Detects ArUco markers in the camera feed
  - Displays the position (x, y, z) of the detected markers
  - Sends the x and y positions via UDP to a specified server address and port
- **Video Saving**:
  - Option to save the video feed as an AVI file with timestamped filenames
- **Stop and Start Acquisition**:
  - The user can start and stop the acquisition process via buttons in the GUI
- **UDP Communication**:
  - Sends ArUco marker position data over UDP to a specified server address

## Requirements:
- PySpin (for camera control)
- OpenCV (for image processing and video handling)
- Tkinter (for GUI)
- ArUco (for marker detection)

## Key Components:
- **GUI**: Built with Tkinter, the app provides fields for FPS, exposure time, and video filename, as well as a checkbox for enabling/disabling video saving and buttons for starting/stopping the acquisition process.
- **Camera Configuration**: The camera is set up with continuous acquisition mode, Mono8 pixel format, and the ability to configure exposure time, gain, and frame rate.
- **ArUco Detection**: The app detects ArUco markers in the video feed, estimates their pose, and draws their positions on the camera feed.
- **UDP Transmission**: The x and y positions of the detected markers are transmitted over UDP for real-time communication.

## Workflow:
1. The user configures the camera settings using the Tkinter GUI.
2. The user starts the acquisition, and the camera begins capturing video.
3. The video is processed for ArUco marker detection, and the position of each marker is displayed.
4. If the user opts to save the video, it is written to an AVI file.
5. The application continuously sends marker positions (x, y) over UDP.
6. The user can stop the acquisition at any time.

In [13]:
import PySpin
import cv2
import tkinter as tk
from tkinter import ttk, filedialog
import threading
import numpy as np
import socket
from datetime import datetime

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Camera Acquisition App")

        # Variables to store camera settings
        self.fps = tk.IntVar(value=30)
        self.exposure = tk.IntVar(value=5000)
        self.output_filename = tk.StringVar(value='output')
        self.save_video = tk.BooleanVar(value=True)

        # Create GUI elements
        self.create_widgets()

        # ArUco detector
        self.detector = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)

        # VideoWriter and CSV file variables
        self.out = None
        self.csv_file = None
        self.stop_acquisition = threading.Event()  # Event to signal stop of acquisition

        # UDP Socket (only needs to be created once)
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.serverAddressPort = ("127.0.0.1", 5053)

    def create_widgets(self):
        # Frame for camera settings
        settings_frame = ttk.LabelFrame(self.root, text="Camera Settings")
        settings_frame.grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)

        # FPS Entry
        ttk.Label(settings_frame, text="FPS:").grid(row=0, column=0, sticky=tk.W)
        fps_entry = ttk.Entry(settings_frame, textvariable=self.fps)
        fps_entry.grid(row=0, column=1, padx=5, pady=5)

        # Exposure Entry
        ttk.Label(settings_frame, text="Exposure (us):").grid(row=1, column=0, sticky=tk.W)
        exposure_entry = ttk.Entry(settings_frame, textvariable=self.exposure)
        exposure_entry.grid(row=1, column=1, padx=5, pady=5)

        # Output Filename Entry
        ttk.Label(settings_frame, text="Output Filename:").grid(row=2, column=0, sticky=tk.W)
        output_entry = ttk.Entry(settings_frame, textvariable=self.output_filename)
        output_entry.grid(row=2, column=1, padx=5, pady=5)

        # Checkbox for video saving
        save_video_checkbox = ttk.Checkbutton(settings_frame, text="Save Video", variable=self.save_video)
        save_video_checkbox.grid(row=7, column=0, columnspan=2, padx=5, pady=5)

        # Start Button
        start_button = ttk.Button(self.root, text="Start Acquisition", command=self.start_acquisition)
        start_button.grid(row=1, column=0, padx=10, pady=10)

        # Stop Button
        stop_button = ttk.Button(self.root, text="Stop Acquisition", command=self.stop_acquisition_process)
        stop_button.grid(row=1, column=1, padx=10, pady=10)

    def browse_output_file(self):
        filename = filedialog.asksaveasfilename(defaultextension=".avi", filetypes=[("AVI files", "*.avi")])
        if filename:
            self.output_filename.set(filename)

    def configure_camera(self, cam):
        nodemap = cam.GetNodeMap()
        
        # Set acquisition mode to continuous
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())
        
        # Set pixel format to Mono8 (8-bit grayscale)
        node_pixel_format = PySpin.CEnumerationPtr(nodemap.GetNode('PixelFormat'))
        node_pixel_format_mono8 = node_pixel_format.GetEntryByName('Mono8')
        node_pixel_format.SetIntValue(node_pixel_format_mono8.GetValue())
        
        # Set maximum width
        node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
        max_width = node_width.GetMax()
        node_width.SetValue(max_width)
        
        # Set maximum height
        node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
        max_height = node_height.GetMax()
        node_height.SetValue(max_height)
        
        # Enable frame rate control
        node_acquisition_frame_rate_enable = PySpin.CBooleanPtr(nodemap.GetNode('AcquisitionFrameRateEnable'))
        node_acquisition_frame_rate_enable.SetValue(True)

        # Set frame rate
        node_frame_rate = PySpin.CFloatPtr(nodemap.GetNode('AcquisitionFrameRate'))
        node_frame_rate.SetValue(self.fps.get())
        
        # Turn off auto exposure
        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
        node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
        
        # Turn off auto gain
        node_gain_auto = PySpin.CEnumerationPtr(nodemap.GetNode('GainAuto'))
        node_gain_auto_off = node_gain_auto.GetEntryByName('Off')
        node_gain_auto.SetIntValue(node_gain_auto_off.GetValue())
        
        # Set exposure time (in microseconds)
        node_exposure_time = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        node_exposure_time.SetValue(self.exposure.get())
        
        # Set gain (if needed)
        node_gain = PySpin.CFloatPtr(nodemap.GetNode('Gain'))
        node_gain.SetValue(10.0)  # Example: set gain to 10 dB

    def acquire_video(self, cam):
        # Generate timestamp string
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        # Initialize VideoWriter if "Save Video" checkbox is checked
        if self.save_video.get():
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            frame_width = int(cam.Width())
            frame_height = int(cam.Height())

            # Add timestamp to the output filename
            filename = f'{timestamp}_{self.output_filename.get()}.avi'
            self.out = cv2.VideoWriter(filename, fourcc, self.fps.get(), (frame_width, frame_height), isColor=False)
            
        else:
            self.out = None

        try:
            # Start the acquisition
            cam.BeginAcquisition()

            while not self.stop_acquisition.is_set():
                # Retrieve next image
                image_result = cam.GetNextImage()

                if image_result.IsIncomplete():
                    print('Image incomplete: ', image_result.GetImageStatus())
                else:
                    # Convert image to numpy array (Grayscale)
                    frame = image_result.GetNDArray()

                    # Process frame for ArUco detection
                    self.detect_aruco(frame)

                # Release image
                image_result.Release()

                # Check for 'Esc' key press to exit (optional)
                if cv2.waitKey(1) == 27:  # Esc key
                    break

        finally:
            # End acquisition
            cam.EndAcquisition()

            # Release VideoWriter and destroy OpenCV window
            if self.out is not None:
                self.out.release()
            cv2.destroyAllWindows()

    def detect_aruco(self, frame):
        # f_x and f_y are focal lengths in pixels
        f_x = (13.5 * 1280) / 6.14  # Focal length in pixels (x-axis)
        f_y = (13.5 * 1024) / 4.92  # Focal length in pixels (y-axis)

        # Define camera parameters (these should be obtained from camera calibration)
        camera_matrix = np.array([[f_x, 0, frame.shape[1] / 2],
                                  [0, f_y, frame.shape[0] / 2],
                                  [0, 0, 1]], dtype=np.float32)
        dist_coeffs = np.zeros((5, 1))  # Assuming no lens distortion

        # Detect ArUco markers
        corners, ids, _ = cv2.aruco.detectMarkers(frame, self.detector)

        if ids is not None and len(ids) > 0:
            # Draw detected markers on the frame
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids, borderColor=(255, 255, 255))

            # Estimate pose of each marker to get tvecs (translation vectors)
            marker_length = 0.05  # Example marker length in meters, adjust accordingly
            rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

            for i in range(len(ids)):
                # Get translation vector (position)
                x, y, z = tvecs[i][0]
                position_text = f"Pos: ({x:.3f}, {y:.3f}, {z:.3f})"

                # Draw the position text in white color
                cv2.putText(frame, position_text, 
                            (int(corners[i][0][0][0]), int(corners[i][0][0][1])),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)  # White color

                # Send only x and y position via UDP
                message = f"{x:.3f},{y:.3f}"
                self.sock.sendto(message.encode(), self.serverAddressPort)

        # Show the frame in OpenCV window
        if self.out is not None:
            self.out.write(frame)  # Write frame to video file
        cv2.imshow("Camera Feed", frame)

    def start_acquisition(self):
        # Start acquisition in a separate thread
        self.stop_acquisition.clear()  # Clear the stop event flag
        acquisition_thread = threading.Thread(target=self.run_acquisition)
        acquisition_thread.start()

    def run_acquisition(self):
        system = PySpin.System.GetInstance()
        cam_list = system.GetCameras()

        try:
            if cam_list.GetSize() == 0:
                raise ValueError("No cameras found!")

            # Assume one camera for simplicity
            cam = cam_list.GetByIndex(0)

            # Initialize camera
            cam.Init()

            # Configure camera settings
            self.configure_camera(cam)

            # Acquire video until stop event is set
            self.acquire_video(cam)

            # Deinitialize camera
            cam.DeInit()

        except PySpin.SpinnakerException as ex:
            print('Error: %s' % ex)
            exit_code = 1

        finally:
            # Release camera and system resources
            del cam
            cam_list.Clear()
            system.ReleaseInstance()

    def stop_acquisition_process(self):
        self.stop_acquisition.clear()

if __name__ == "__main__":
    root = tk.Tk()
    app = CameraApp(root)
    root.mainloop()
